In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from collections import Counter
from itertools import product
from itertools import combinations
import matplotlib.pyplot as plt
import openpyxl


In [2]:

def TransitionCounts(T, M):
    pairs: list[str] = [str(T[i]) + str(T[i + 1]) for i in range(len(T) - 1)]
    pairs_order = [str(i) + str(j) for i, j in product(range(M), repeat=2)]
    iter = Counter(pairs)
    iter = [iter[pair] for pair in pairs_order]
    return iter

def mle(N):
    p_ij_hat_matrix = N / N.sum(axis=1, keepdims=True)
    return p_ij_hat_matrix

def prob_diff(z):
    pairs= list(combinations(z,2))
    w=[]
    for pair in pairs:
        diff = abs(pair[0]-pair[1])
        w.append(diff)
    return np.array(w)

def penalty_function(numerator,denomiator):  
    pelt = numerator/denomiator
    pelt[denomiator==0]=0 
    return pelt


def eval_g_eq(x):
    row = [x[i*M:(i+1)*M] for i in range(len(x)//M)]
    constraints = []
    for z in row:
        constraints.extend([np.sum(z)-1])
    return constraints


def ada_lasso(data, lam):
    n =TransitionCounts(data,M)
    n_ij_matrix = np.reshape(n,(M,M))
    p_ij_hat_matrix = mle(n_ij_matrix)
    p_ij_hat = p_ij_hat_matrix.flatten()  
    w_deno= prob_diff(p_ij_hat) 
    def obj_f(x): #### Objective function
        w_num  = prob_diff(x) 
        pelt = penalty_function(w_num,w_deno)
        return -np.sum((n * np.log(x))) + lam*np.sum(pelt) 
    
    res = minimize(obj_f, p_ij_hat, 
    constraints={'type': 'eq', 'fun': eval_g_eq}, 
    bounds=[(0, 1)] * (M**2), 
    method='SLSQP', 
    options={'maxiter': 160000}
    )
    p_tilde = res.x
    return n, p_tilde



def lasso(data, lam):
    n =TransitionCounts(data,M)
    n_ij_matrix = np.reshape(n,(M,M))
    p_ij_hat_matrix = mle(n_ij_matrix)
    p_ij_hat = p_ij_hat_matrix.flatten()  
    w_deno= prob_diff(p_ij_hat) 
    def obj_f(x): 
        diff_x  = prob_diff(x) 
        return -np.sum((n * np.log(x))) + lam*np.sum(diff_x)
    
    res = minimize(obj_f, p_ij_hat, 
    constraints={'type': 'eq', 'fun': eval_g_eq}, 
    bounds=[(0, 1)] * (M**2), 
    method='SLSQP', 
    options={'maxiter': 160000}
    )
    p_check = res.x
    return n, p_check


def purity(tilde):
    purity_set = []
    each_class_tilde_index = []
    class_tilde = np.unique(tilde)
    each_class_tilde_index = []
    for class_val in class_tilde:
        indices = np.where(tilde == class_val)[0]
        each_class_tilde_index.append(indices.tolist())
    max_count = []
    for ell in each_class_index:
        count = []
        for i in each_class_tilde_index:
            count.append(len(np.intersect1d(ell,i)))
        max_count.append(np.max(count))
    purity_result = np.sum(max_count)/M**2    
    return purity_result


def FDnorm(b):
    b_matrix = np.reshape(b,(M,M))
    p_ij_tilde_bar = b_matrix- p_ij_true
    norm = np.sqrt(np.sum(np.abs(p_ij_tilde_bar) ** 2))
    return norm



In [ ]:
cv_each_lambda_adalasso=[]
for j in range(len(LAMBDA_ada)):
    L = LAMBDA_ada[j]
    cv_each_training_adalasso = []
    cv_each_test_adalasso=[]
    cv_each_training_lasso = []
    cv_each_test_lasso=[]
    for i in range(T):
        testing_set = DATA[i]
        n_ij_testing = TransitionCounts(testing_set,M)
        training_set = np.delete(DATA, i, axis=0)
        for z in range(T-1):
            train_subdata = training_set[z]
            n_train, p_tilde_train = ada_lasso(train_subdata,L)
            cv_adalasso = -np.sum(n_ij_testing*np.log(p_tilde_train))  
            cv_each_training_adalasso.append(np.sum(cv_adalasso))
            
        cv_each_test_adalasso.append(np.sum(cv_each_training_adalasso))
    cv_each_lambda_adalasso.append(np.sum(cv_each_test_adalasso))    


cv_each_lambda_lasso = []

for j in range(len(LAMBDA_la)):
    L = LAMBDA_la[j]
    cv_each_training_lasso = []
    cv_each_test_lasso=[]
    for i in range(T):
        testing_set = DATA[i]
        n_ij_testing = TransitionCounts(testing_set,M)
        training_set = np.delete(DATA, i, axis=0)
        for z in range(T-1):
            train_subdata = training_set[z]
            n_train_lasso, p_tilde_train_lasso = lasso(train_subdata,L)
            cv_lasso = -np.sum(n_ij_testing*np.log(p_tilde_train_lasso))  
            cv_each_training_lasso.append(np.sum(cv_lasso))
        cv_each_test_lasso.append(np.sum(cv_each_training_lasso))
    cv_each_lambda_lasso.append(np.sum(cv_each_test_lasso))   

